# Grab select Census Tracts

In [ ]:
#Import the GIS object and authenticate
from arcgis import GIS
gis = GIS('pro')

In [ ]:
#Use the 'content' helper to search for feature layers with keyword Census
results = gis.content.search(query='census  tract',item_type='Feature Layer',outside_org=True)
results

In [ ]:
#Have a look at the 4th result
results[3]

In [ ]:
#Save it as an arcgis "item" object
tractsItem = results[3]
type(tractsItem)

In [ ]:
#Get the layer from the item
tractsLyr = tractsItem.layers[0]
type(tractsLyr)

Query the feature layer: <br>
https://developers.arcgis.com/python/guide/working-with-feature-layers-and-features/#Querying-feature-layers

In [ ]:
#List the fields associated with the raster
for f in tractsLyr.properties.fields:
    print(f['name'])

In [ ]:
query_result = tractsLyr.query("FIPS LIKE '37063%'")
len(query_result)

In [ ]:
#View results as a spatially enabled data frame
sdf = query_result.sdf
sdf.head()

In [ ]:
sdf['SHAPE'][0]['rings']

In [ ]:
#Convert rings to polygons
sdf['geometry'] = sdf['SHAPE'].apply(lambda x: x['rings'])
sdf.head()

In [ ]:
# This is a function we can use to convert ESRI rings to Shapely polygons
from shapely.geometry import Polygon
from shapely.geometry import LinearRing
def polyFromRing(ring):
    r = LinearRing(ring)
    s = Polygon(r)
    return r

In [ ]:
#Apply the function to all rows
sdf['geometry']=sdf['SHAPE'].apply(lambda x: Polygon(x.rings[0]))

In [ ]:
#Drop the old "SHAPE" field
import geopandas as gpd
sdf.drop('SHAPE',axis=1,inplace=True)

In [ ]:
#Convert the dataframe to a geopandas dataframe
gdf=gpd.GeoDataFrame(sdf)

In [ ]:
#Export to a shapefile
gdf.to_file(driver='ESRI Shapefile',filename='foo.shp')